In [13]:
from openai import OpenAI
from anthropic import Anthropic

from dotenv import load_dotenv

import base64, os

In [4]:
image_path = "C:\\test_test\\response_rate_of_image_description\\image.png"

GPT

In [8]:
from pydantic import BaseModel, Field

class ImageDescription(BaseModel):
    category : str = Field(description = "이미지의 카테고리. 4가지 카테고리 : ['figure', 'chart', 'table', 'equation']")
    description : str = Field(description = "이미지의 설명")

In [6]:
def encode_image(image_path : str):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [9]:
client = OpenAI()

base64_image = encode_image(image_path)

ext = os.path.splitext(image_path)[1].replace('.','')

message_list = [
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "주어진 이미지를 분석하세요. 이미지 종류('figure', 'chart', 'table', 'equation')와 설명을 반환하세요. 표의 경우 담겨있는 정보를 생략하지 말고 전부 일목요연하게 제공하세요. 설명의 경우 마크다운으로 작성하세요요"
                },
                {
                    "type": "input_image",
                    "image_url": f"data:image/{ext};base64,{base64_image}",
                    "detail": "high"
                }
            ]
        }
    ]

In [10]:
response = client.responses.parse(
        model = "gpt-4.1",
        input = message_list,
        text_format=ImageDescription
    )

In [11]:
print(response.output_parsed)

category='figure' description='이 이미지는 만화 스타일로 그려진, 활짝 웃으며 춤을 추는 듯한 하마를 묘사하고 있습니다. 하마는 밝은 색조와 부드러운 터치로 표현되어 있으며, 배경은 따뜻한 색상의 식물 무늬로 이루어져 부드럽고 아늑한 분위기를 연출합니다. 하마는 한쪽 다리를 들고 두 팔을 크게 벌리며 환하게 웃고 있어, 유쾌하고 생동감 넘치는 모습을 나타냅니다.'


Anthropic

In [14]:
client = Anthropic()

In [15]:
message_list = [
    {
        "role" : "user",
        "content" : [
            {
                "type" : "image",
                "source" : {
                    "type" : "base64",
                    "media_type" : "image/png",
                    "data" : base64_image
                }
            },
            {
                "type" : "text",
                "text" : "Describe this image."
            }
        ]
    }
]

In [16]:

message = client.messages.create(
    model = "claude-sonnet-4-20250514",
    max_tokens = 1024,
    messages =message_list
)

In [17]:
print(message)

Message(id='msg_01A9uAhUfQaYS2sJhd9MTC8v', content=[TextBlock(citations=None, text="This is a charming cartoon illustration of a cheerful hippopotamus in a dynamic, joyful pose. The hippo appears to be dancing or celebrating, with its arms outstretched and mouth wide open in what looks like laughter or singing. The character has a round, pudgy body typical of cartoon hippos, with small ears and a big smile showing its teeth.\n\nThe artwork is rendered in warm, golden-brown tones that give it a cozy, friendly atmosphere. The hippo is depicted against a textured background with swirling patterns that complement the warm color palette. The illustration style is smooth and polished, with soft shading and highlights that give the character dimension and personality.\n\nThe overall mood of the image is very positive and energetic, capturing a sense of happiness and exuberance through the hippo's animated pose and expression.", type='text')], model='claude-sonnet-4-20250514', role='assistant'

Gemini

In [20]:
pip install google-genai

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled httpx-0.27.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.128 requires langchain-openai<0.3.0,>=0.2.1, but you have langchain-openai 0.3.14 which is incompatible.
litellm 1.60.2 requires httpx<0.28.0,>=0.23.0, but you have httpx 0.28.1 which is incompatible.


In [22]:
from google.genai import types
from google import genai

In [23]:
client = genai.Client()

In [28]:
response = client.models.generate_content(
    model = "gemini-2.5-flash-preview-04-17",
    contents = [
        types.Part.from_bytes(
            data = base64_image,
            mime_type = "image/png"
        ),
        'Describe this image.'
    ]
)

In [29]:
print(response.text)

A charming illustration depicts a chubby cartoon hippo joyfully dancing. The hippo, rendered in warm shades of brown with visible sketch lines, is balanced on one leg, with the other lifted slightly. Its arms are wide open and raised high, as if mid-leap or twirl. The hippo's head is tilted back slightly, eyes closed with a beaming, wide-open mouth revealing its teeth and tongue, conveying a sense of pure happiness and abandon. A tiny tail is visible in the back. The background is a soft, muted wash of earthy tones, including yellows, browns, and hints of green, with faint, blurry, sketchy shapes suggesting abstract foliage or a natural, organic setting. The overall style is whimsical and cheerful, reminiscent of a children's book illustration.


In [30]:
response = client.models.generate_content(
    model = "gemini-2.0-flash",
    contents = [
        types.Part.from_bytes(
            data = base64_image,
            mime_type = "image/png"
        ),
        'Describe this image.'
    ]
)

In [31]:
print(response.text)

Here is a description of the image:

A cartoon hippopotamus is happily dancing. It is brown and stands on its left leg with the right leg raised slightly. The hippo has both arms raised as well. It has a wide open mouth with visible teeth, and closed eyes. The background is a soft, blurred mix of yellows, browns, and greens, suggesting a natural or forest-like environment. The style is reminiscent of a digital painting with textured shading.


Gemini GPT 스타일일

In [32]:
client = OpenAI(
    api_key = os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [33]:
message_list = [
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "주어진 이미지를 분석하세요. 이미지 종류('figure', 'chart', 'table', 'equation')와 설명을 반환하세요. 표의 경우 담겨있는 정보를 생략하지 말고 전부 일목요연하게 제공하세요. 설명의 경우 마크다운으로 작성하세요요"
                },
                {
                    "type": "input_image",
                    "image_url": f"data:image/{ext};base64,{base64_image}",
                    "detail": "high"
                }
            ]
        }
    ]

In [39]:
response = client.chat.completions.create(
  model="gemini-2.5-flash-preview-04-17",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is in this image?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)


In [40]:
print(response.choices[0].message.content)

The image contains a cartoon-style drawing of a cheerful hippo. The hippo is standing on one leg, with its arms outstretched and a wide open, laughing mouth, suggesting it is dancing or cheering happily. The background is a warm, sketched style.


XAI

In [41]:
XAI_API_KEY = os.getenv("XAI_API_KEY")

In [42]:
client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)

In [49]:
response = client.chat.completions.create(
  model="grok-2-vision-latest",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is in this image?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)

In [47]:
print(response.choices[0].message.content)

The image depicts a cartoon-style hippopotamus with a joyful expression. The hippo is standing on one leg with its arms raised, appearing to be in a happy or celebratory pose. The background is a warm, earthy tone, suggesting a natural or jungle setting. The overall mood of the image is cheerful and lighthearted.


Mistral

In [53]:
from mistralai import Mistral

In [54]:
client = Mistral()

In [55]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "What's in this image?"
            },
            {
                "type": "image_url",
                "image_url": f"data:image/jpeg;base64,{base64_image}" 
            }
        ]
    }
]

In [58]:
response = client.chat.complete(
    model = "pixtral-12b-2409",
    messages = message_list
)



ValidationError: 3 validation errors for Unmarshaller
body.0.user.content.str
  Input should be a valid string [type=string_type, input_value=[{'type': 'text', 'text':...6W1uAAAAAElFTkSuQmCC'}}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
body.0.user.content.list[tagged-union[ImageURLChunk,DocumentURLChunk,TextChunk,ReferenceChunk]].1.image_url.image_url.ImageURL.url
  Field required [type=missing, input_value={'image_url': 'data:image...Nf6W1uAAAAAElFTkSuQmCC'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
body.0.user.content.list[tagged-union[ImageURLChunk,DocumentURLChunk,TextChunk,ReferenceChunk]].1.image_url.image_url.str
  Input should be a valid string [type=string_type, input_value={'image_url': 'data:image...Nf6W1uAAAAAElFTkSuQmCC'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type